#SGD with lr = 0.05

In [15]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])




class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.c2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.c3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.c1(x)
        x = self.max_pool(self.relu(x))
        x = self.c2(x)
        x = self.max_pool(self.relu(x))
        x = self.relu(self.c3(x))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
  
        optimizer.zero_grad()
        output = model(data)
        criterion = nn.CrossEntropyLoss()
            
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
    
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    epochs = 50
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    #torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301488
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.899701
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.262967
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.911716
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.068746
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.712040
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.604024
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.368663
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.286739
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.270055
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.331538
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.431964
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.178124
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.266083
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.205457
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.209571
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.191637
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.193267
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.129404
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.08

#SGD with lr = 0.0001

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])




class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.c2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.c3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.c1(x)
        x = self.max_pool(self.relu(x))
        x = self.c2(x)
        x = self.max_pool(self.relu(x))
        x = self.relu(self.c3(x))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
       
        optimizer.zero_grad()
        output = model(data)
        criterion = nn.CrossEntropyLoss()
            
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    epochs = 50
    lr = 0.0001
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    #torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306236
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.291103
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.293686
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.298812
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.290999
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.306493
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.301476
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.300013
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.292499
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.295002
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.301082
Train Epoch: 1 [14080/60000 (23%)]	Loss: 2.306349
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.297397
Train Epoch: 1 [16640/60000 (28%)]	Loss: 2.296515
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.296537
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.306605
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.301216
Train Epoch: 1 [21760/60000 (36%)]	Loss: 2.293568
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.316096
Train Epoch: 1 [24320/60000 (41%)]	Loss: 2.30

#Adagrad with lr = 0.05

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])




class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.c2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.c3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.c1(x)
        x = self.max_pool(self.relu(x))
        x = self.c2(x)
        x = self.max_pool(self.relu(x))
        x = self.relu(self.c3(x))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        criterion = nn.CrossEntropyLoss()
            
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
       
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    epochs = 50
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    #torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.Adagrad(model.parameters(), lr=lr,  weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..


100%|██████████| 9912422/9912422 [00:00<00:00, 43855264.01it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 69939777.03it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27282591.48it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13415865.33it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295886
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.307098
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.277824
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.296020
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.309189
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.315057
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.307776
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.308193
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.284404
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.299447
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.307589
Train Epoch: 1 [14080/60000 (23%)]	Loss: 2.306442
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.301824
Train Epoch: 1 [16640/60000 (28%)]	Loss: 2.306098
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.295653
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.300780
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.294034
Train Epoch: 1 [21760/60000 (36%)]	Loss: 2.304585
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.294132
Train Epoch: 1 [24320/60000 (41%)]	Loss: 2.308712
Train Epoch: 1 [

#Adagrad with lr = 0.0001

In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])




class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.c2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.c3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.c1(x)
        x = self.max_pool(self.relu(x))
        x = self.c2(x)
        x = self.max_pool(self.relu(x))
        x = self.relu(self.c3(x))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
      
        optimizer.zero_grad()
        output = model(data)
        criterion = nn.CrossEntropyLoss()
            
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
       
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    epochs = 50
    lr = 0.0001
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    #torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.Adagrad(model.parameters(), lr=lr,  weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308893
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.285639
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.292954
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.292593
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.290761
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.305504
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.291087
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.281800
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.290568
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.278767
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.292284
Train Epoch: 1 [14080/60000 (23%)]	Loss: 2.267730
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.277172
Train Epoch: 1 [16640/60000 (28%)]	Loss: 2.271443
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.275042
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.274359
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.283195
Train Epoch: 1 [21760/60000 (36%)]	Loss: 2.277884
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.267168
Train Epoch: 1 [24320/60000 (41%)]	Loss: 2.27

#RMSprop with lr = 0.05

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])




class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.c2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.c3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.c1(x)
        x = self.max_pool(self.relu(x))
        x = self.c2(x)
        x = self.max_pool(self.relu(x))
        x = self.relu(self.c3(x))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        criterion = nn.CrossEntropyLoss()
            
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    epochs = 50
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305028
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.396305
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.299181
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.329215
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.286761
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.322347
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.302972
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.305793
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.342294
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.315349
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.300236
Train Epoch: 1 [14080/60000 (23%)]	Loss: 2.314507
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.299644
Train Epoch: 1 [16640/60000 (28%)]	Loss: 2.296975
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.302947
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.314002
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.318245
Train Epoch: 1 [21760/60000 (36%)]	Loss: 2.294972
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.305797
Train Epoch: 1 [24320/60000 (41%)]	Loss: 2.32

#RMSprop with lr = 0.0001


In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.1307,), std = (0.3081,)),
])




class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.c1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.c2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.c3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.c1(x)
        x = self.max_pool(self.relu(x))
        x = self.c2(x)
        x = self.max_pool(self.relu(x))
        x = self.relu(self.c3(x))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
       
        optimizer.zero_grad()
        output = model(data)
        criterion = nn.CrossEntropyLoss()
            
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
     
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    epochs = 50
    lr = 0.0001
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    #torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

==> Preparing data..
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306866
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.192313
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.948859
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.719480
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.436721
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.339350
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.176858
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.975178
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.974290
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.776342
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.850468
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.632197
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.628873
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.666667
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.570366
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.590237
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.583213
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.627853
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.511940
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.48